In [ ]:
import numpy as np
from tqdm import tqdm
import pickle
import time
from scipy.special import logsumexp
from scipy.stats import multivariate_normal, multivariate_t, random_correlation
from matplotlib import pyplot as plt
import yaml
import os
import imageio
import moviepy.editor as mp


In [ ]:
def save(obj, filename):
    """Save compiled models for reuse."""
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    """Reload compiled models for reuse."""
    return pickle.load(open(filename, 'rb'))

In [ ]:
def plot_contour_lines(dist1, dist2, iteration=0):
    # Generate a grid of points
    x = np.linspace(-10, 10, 100)
    y = np.linspace(-10, 10, 100)
    X, Y = np.meshgrid(x, y)
    positions = np.vstack([X.ravel(), Y.ravel()])

    # Calculate the probability density for each distribution at each point on the grid
    Z1 = dist1.pdf(positions.T)
    Z2 = dist2.pdf(positions.T)

    # Reshape the probability density values to match the grid shape
    Z1 = Z1.reshape(X.shape)
    Z2 = Z2.reshape(X.shape)

    # Create a new figure and axis
    fig, ax = plt.subplots()

    # Plot the contour lines for the first distribution in blue color
    ax.contour(X, Y, Z1, colors='blue', label='Proposal')

    # Plot the contour lines for the second distribution in red color
    ax.contour(X, Y, Z2, colors='red', label='Target')

    # Set axis labels and title
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_title('Contour Lines')

    # # Add a legend in the upper right corner
    # ax.legend()

    plt.savefig("results/ais-heavy-iter-{}.png".format(iteration), bbox_inches="tight", dpi=100)
    # plt.show()

In [ ]:
# Given log weights, return the normalized weights
def normalize_log(log_weights):
	return np.exp(log_weights - logsumexp(log_weights)).flatten()

# Given log weights, return the * log of * the normalized weights
def log_normalize_log(log_weights):
	return log_weights - logsumexp(log_weights)


In [ ]:
# Load any settings
with open("settings.yaml", mode="r") as file:
    settings = yaml.safe_load(file)


In [ ]:
np.random.seed(0)
D = settings['D']
ddof_target = settings['ddof_target']
ddof_proposal = ddof_target

In [ ]:
# Target
mean_target = np.zeros(D)
std_devs_target = np.diag(np.sqrt(np.ones(D)*2))
random_corr_mat = random_correlation(eigs=np.ones(D)).rvs(1)
cov_target =  std_devs_target @ random_corr_mat  @ std_devs_target
shape_target = ((ddof_target - 2) / ddof_target) * cov_target

# Proposal
mean_proposal = np.ones(D) * 0.25
std_devs_proposal = np.diag(np.sqrt(np.ones(D)*4))
# random_corr_mat = random_correlation(eigs= np.ones(D)).rvs(1)
cov_proposal = std_devs_proposal @ random_corr_mat @ std_devs_proposal

shape_proposal = ((ddof_proposal - 2) / ddof_proposal) * cov_proposal

In [ ]:
def AMIS(mu_initial,shape_initial, n_iterations, target_pdf, ddof_proposal, M=5000):
    all_normalized_logweights = np.empty((n_iterations,M))
    all_logweights = np.empty((n_iterations,M))
    proposals_over_iterations = []

    # Iteration 0
    first_proposal = multivariate_t(loc=mu_initial,shape=shape_initial,df=ddof_proposal)
    proposals_over_iterations.append(first_proposal)
    samples_initial = first_proposal.rvs(size=M)

    log_numerator = target_pdf.logpdf(samples_initial)
    log_denominator = first_proposal.logpdf(samples_initial)

    # assert log_numerator.shape == log_denominator.shape

    all_logweights[0,:] = log_numerator - log_denominator
    all_normalized_logweights[0,:] = log_normalize_log(log_numerator - log_denominator)

    mu_current, shape_current =  np.average(samples_initial, weights=np.exp(all_normalized_logweights[0,:]), axis=0), ((ddof_target - 2) / ddof_target) * np.cov(samples_initial, rowvar=False, aweights=np.exp(all_normalized_logweights[0,:]))

    # assert np.allclose( np.average(samples_initial, weights=np.exp(all_normalized_logweights[0,:]),axis=0), np.sum( np.prod( samples_initial, np.exp(all_normalized_logweights[0,:]) ) , axis=0) )
    # assert mu_current.shape == (D,) and shape_current.shape == (D,D)

    # Iteration t > 0
    for t in tqdm(range(1,n_iterations)):

        current_proposal = multivariate_t(loc=mu_current,shape=shape_current,df=ddof_proposal)

        proposals_over_iterations.append(current_proposal)

        # Plot current proposal vs target
        plot_contour_lines(current_proposal,target_pdf,iteration=t)

        # Draw M samples from current proposal
        samples_current = current_proposal.rvs(size=M)

        # Weighting and re-weighting procedure
        # Numerator
        log_numerator = target_pdf.logpdf(samples_current)
        # Note the mixture in the denominator !
        mixture_denominator_evaluations = np.asarray([ proposals_over_iterations[prev_t].logpdf(samples_current) for prev_t in range(0,t+1) ])

        log_denominator = - np.log(t) + logsumexp( mixture_denominator_evaluations, axis=0) # check correct axis

        # assert log_numerator.shape == log_denominator.shape

        all_logweights[t,:] = log_numerator - log_denominator
        all_normalized_logweights[t,:] = log_normalize_log(log_numerator - log_denominator)

        # Update proposal
        mu_current, shape_current = np.average(samples_current, weights=np.exp(all_normalized_logweights[t,:]), axis=0), ((ddof_target - 2) / ddof_target) * np.cov(samples_current, rowvar=False, aweights=np.exp(all_normalized_logweights[t,:]))

#         print(mu_current - mean_target)
#         print(shape_current - shape_target)

In [ ]:
plot_contour_lines(multivariate_t(loc=mean_proposal,shape=shape_proposal,df=ddof_proposal), multivariate_t(loc=mean_target,shape=shape_target,df=ddof_proposal), iteration=0)

In [ ]:
AMIS(mu_initial=mean_proposal,shape_initial=shape_proposal, n_iterations=150, target_pdf=multivariate_t(loc=mean_target,shape=shape_target,df=ddof_target), ddof_proposal=ddof_proposal)

In [ ]:
# Write movie to a file
filenames = [os.path.join("results", "ais-heavy-iter-{}.png".format(i)) for i in range(0,150)]

with imageio.get_writer(os.path.join("results", 'movie.gif'), mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

clip = mp.VideoFileClip(os.path.join("results", 'movie.gif'))
clip.write_videofile(os.path.join("results", 'movie.mp4'))
